In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import os
import sys 
sys.path.append('../')
from more_itertools import chunked
import csv
from gobancv.stones import draw_circles

def read_labels(filename):
    files = []
    labels = []
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        next(reader) # skip header
        for row in reader:
            files.append(row[0])
            labels.append(list(chunked(map(int, row[1:]), 2)))
    return files, labels

def imshow(img, size=(5, 5)):
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    plt.rcParams['figure.figsize'] = size
    plt.imshow(img)
    plt.yticks([])
    plt.xticks([])
    plt.show()

files, labels = read_labels("../data/labels/border_labels.csv")
imgs = []
for file in files:
    imgs.append(cv.imread("../" + file))

warped = []
for img, label in zip(imgs, labels):
    W = min(img.shape[:2])
    imgc = img.copy()

    dest_points = np.array([[0, 0], [0, W], [W, W], [W, 0]], dtype=np.float32)
    M = cv.getPerspectiveTransform(np.array(label, dtype=np.float32), dest_points)
    
    img_warped = cv.warpPerspective(imgc, M, (W, W))
    warped.append(img_warped)

## Median blur

In [ ]:
for warp in warped:
    gray = cv.cvtColor(warp, cv.COLOR_BGR2GRAY)
    # mean blur
    blur = cv.medianBlur(gray, 13)
    circles = cv.HoughCircles(
        blur,
        cv.HOUGH_GRADIENT,
        dp=1,
        minDist=10,
        param1=200, # upper threshold for Canny edge detector
        param2=15,
        minRadius=5,
        maxRadius=50
    )
    with_circles = gray.copy()
    draw_circles(with_circles, circles)
    imshow(np.concatenate([gray, blur, with_circles], axis=1), size=(15, 5))